In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision

import PIL

import numpy as np
import matplotlib.pyplot as plt

import os
import re

In [2]:
f_path = "data/labeldata2/"

In [3]:
IMAGE_SIZE = 28

In [4]:
class TangramImages(Dataset):
    def __init__(self, folder_path,  transforms=None):
        self.data_root_folder = folder_path
        self.image_list  = self.load_image_list(self.data_root_folder)
        self.transforms = transforms
        
    def load_image_list(self, folder_path):
        image_list = []
        for time_folder in os.listdir(folder_path):
            image_folder = os.path.join(folder_path, time_folder)
            for file_name in os.listdir(image_folder):
                image_list.append(os.path.join(image_folder, file_name))
        
        return image_list
    
    def __len__(self):
        return len(self.image_list)
    
    def __getitem__(self, index):
        single_image_path = self.image_list[index]
        # Open image
        img = PIL.Image.open(single_image_path).convert("L")
        img = PIL.ImageOps.invert(img)
        #img = resize(img,(IMAGE_SIZE,IMAGE_SIZE))
        if self.transforms is not None:
            img = self.transforms(img)
            
        score = float(re.split(r"(\\|\.|/)",single_image_path)[-3])
        return img, score / 8.0
        

In [5]:
transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((IMAGE_SIZE,IMAGE_SIZE), interpolation = PIL.Image.NEAREST),
    torchvision.transforms.RandomAffine(degrees = 90, translate = (0.2,0.2), scale = (0.6,1)),
    torchvision.transforms.ColorJitter(hue=.05, saturation=.05),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
    # torchvision.transforms.RandomRotation(20, resample=PIL.Image.BILINEAR)
    torchvision.transforms.ToTensor(),
])

In [6]:
ti = TangramImages(f_path, transforms)

In [7]:
ti[30][0].shape

torch.Size([1, 28, 28])

In [8]:
torch.max(ti[30][0])

tensor(1.)

In [9]:
import torch.nn as nn
from torch import Tensor

In [10]:
# from typing import Type, Any, Callable, Union, List, Optional

In [11]:
# from torchvision.models.resnet import BasicBlock, Bottleneck, conv1x1, conv3x3

In [12]:
# class ResNet(nn.Module):
#     def __init__(
#         self,
#         block: Type[Union[BasicBlock, Bottleneck]],
#         layers: List[int],
#         num_classes: int = 1000,
#         zero_init_residual: bool = False,
#         groups: int = 1,
#         width_per_group: int = 64,
#         replace_stride_with_dilation: Optional[List[bool]] = None,
#         norm_layer: Optional[Callable[..., nn.Module]] = None
#     ) -> None:
#         super(ResNet, self).__init__()
#         if norm_layer is None:
#             norm_layer = nn.BatchNorm2d
#         self._norm_layer = norm_layer

#         self.inplanes = 64
#         self.dilation = 1
#         if replace_stride_with_dilation is None:
#             # each element in the tuple indicates if we should replace
#             # the 2x2 stride with a dilated convolution instead
#             replace_stride_with_dilation = [False, False, False]
#         if len(replace_stride_with_dilation) != 3:
#             raise ValueError("replace_stride_with_dilation should be None "
#                              "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
#         self.groups = groups
#         self.base_width = width_per_group
#         #change 3 -> 1 for gray scale
#         self.conv1 = nn.Conv2d(1, self.inplanes, kernel_size=7, stride=2, padding=3,
#                                bias=False)
#         self.bn1 = norm_layer(self.inplanes)
#         self.relu = nn.ReLU(inplace=True)
#         self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
#         self.layer1 = self._make_layer(block, 64, layers[0])
#         self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
#                                        dilate=replace_stride_with_dilation[0])
#         self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
#                                        dilate=replace_stride_with_dilation[1])
#         self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
#                                        dilate=replace_stride_with_dilation[2])
#         self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
#         self.fc = nn.Linear(512 * block.expansion, num_classes)

#         for m in self.modules():
#             if isinstance(m, nn.Conv2d):
#                 nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
#             elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
#                 nn.init.constant_(m.weight, 1)
#                 nn.init.constant_(m.bias, 0)

#         # Zero-initialize the last BN in each residual branch,
#         # so that the residual branch starts with zeros, and each residual block behaves like an identity.
#         # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
#         if zero_init_residual:
#             for m in self.modules():
#                 if isinstance(m, Bottleneck):
#                     nn.init.constant_(m.bn3.weight, 0)  # type: ignore[arg-type]
#                 elif isinstance(m, BasicBlock):
#                     nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]

#     def _make_layer(self, block: Type[Union[BasicBlock, Bottleneck]], planes: int, blocks: int,
#                     stride: int = 1, dilate: bool = False) -> nn.Sequential:
#         norm_layer = self._norm_layer
#         downsample = None
#         previous_dilation = self.dilation
#         if dilate:
#             self.dilation *= stride
#             stride = 1
#         if stride != 1 or self.inplanes != planes * block.expansion:
#             downsample = nn.Sequential(
#                 conv1x1(self.inplanes, planes * block.expansion, stride),
#                 norm_layer(planes * block.expansion),
#             )

#         layers = []
#         layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
#                             self.base_width, previous_dilation, norm_layer))
#         self.inplanes = planes * block.expansion
#         for _ in range(1, blocks):
#             layers.append(block(self.inplanes, planes, groups=self.groups,
#                                 base_width=self.base_width, dilation=self.dilation,
#                                 norm_layer=norm_layer))

#         return nn.Sequential(*layers)

#     def _forward_impl(self, x: Tensor) -> Tensor:
#         # See note [TorchScript super()]
#         x = self.conv1(x)
#         x = self.bn1(x)
#         x = self.relu(x)
#         x = self.maxpool(x)

#         x = self.layer1(x)
#         x = self.layer2(x)
#         x = self.layer3(x)
#         x = self.layer4(x)

#         x = self.avgpool(x)
#         x = torch.flatten(x, 1)
#         x = self.fc(x)

#         return x

#     def forward(self, x: Tensor) -> Tensor:
#         return self._forward_impl(x)


In [13]:
##model = ResNet(BasicBlock, [1,1,1,1])

In [14]:
#ti_dataloader = DataLoader(ti, batch_size=4,
#                        shuffle=True, num_workers=0)

In [15]:
# for batch_input, batch_label in ti_dataloader:
#    print(batch_input, batch_label)
#    break

In [16]:
# batch_input.shape

In [17]:
#model(batch_input)

In [18]:
def conv3x3(in_channels, out_channels, **kwargs):
    # The convolutional layers (for feature extraction) use standard layers from
    # `torch.nn`, since they do not require adaptation.
    # See `examples/maml/model.py` for comparison.
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, **kwargs),
        nn.BatchNorm2d(out_channels, momentum=1., track_running_stats=False),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

In [19]:
class ConvolutionalNeuralNetwork(nn.Module):
    def __init__(self, in_channels, out_features, hidden_size=64):
        super(ConvolutionalNeuralNetwork, self).__init__()
        self.in_channels = in_channels
        self.out_features = out_features
        self.hidden_size = hidden_size

        self.features = nn.Sequential(
            conv3x3(in_channels, hidden_size),
            conv3x3(hidden_size, hidden_size),
            conv3x3(hidden_size, hidden_size),
            conv3x3(hidden_size, hidden_size)
        )

        # Only the last (linear) layer is used for adaptation in ANIL
        self.linear = nn.Sequential(
            nn.Linear(hidden_size, out_features),
            nn.Sigmoid(),
        )

    def forward(self, inputs, params=None):
        features = self.features(inputs)
        #print(features.shape)
        features = features.view((features.size(0), -1))
        #print(features.shape)
        scores = self.linear(features)
        return scores

In [20]:
model = ConvolutionalNeuralNetwork(1, 1, hidden_size=64)

In [21]:
if torch.cuda.is_available():
    model = model.cuda()

In [22]:
# model(batch_input)

In [23]:
train_dataset_len = int(0.95 * len(ti))
test_dataset_len = len(ti) - train_dataset_len

In [24]:
train_dataset, test_dataset = torch.utils.data.random_split(ti,[train_dataset_len, test_dataset_len])

In [25]:
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=True, num_workers=0)

In [26]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [27]:
epochs = 100

In [28]:
for epoch in range(epochs):
    loss_epoch = []
    count = 0
    for batch_inputs, batch_scores in train_dataloader:
        count += 1
        #print(batch_inputs.shape)
        if torch.cuda.is_available():
            batch_inputs = batch_inputs.to("cuda")
            batch_scores = batch_scores.to("cuda")
        
        pred_scores = model(batch_inputs).view(-1)
        
        loss = torch.sum((pred_scores - batch_scores)**2)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss_epoch.append(loss.item())
        
        # if count % 20 == 0:
        #    print(loss)
        
    print("epoch: {} avg loss {:.2f}".format(epoch, sum(loss_epoch)/(len(loss_epoch) + 1e-3)))

epoch: 0 avg loss 0.70
epoch: 1 avg loss 0.45
epoch: 2 avg loss 0.42
epoch: 3 avg loss 0.42
epoch: 4 avg loss 0.39
epoch: 5 avg loss 0.41
epoch: 6 avg loss 0.37
epoch: 7 avg loss 0.39
epoch: 8 avg loss 0.37
epoch: 9 avg loss 0.37
epoch: 10 avg loss 0.37
epoch: 11 avg loss 0.37
epoch: 12 avg loss 0.36
epoch: 13 avg loss 0.34
epoch: 14 avg loss 0.33
epoch: 15 avg loss 0.34
epoch: 16 avg loss 0.35
epoch: 17 avg loss 0.34
epoch: 18 avg loss 0.34
epoch: 19 avg loss 0.33
epoch: 20 avg loss 0.34
epoch: 21 avg loss 0.33
epoch: 22 avg loss 0.33
epoch: 23 avg loss 0.33
epoch: 24 avg loss 0.31
epoch: 25 avg loss 0.33
epoch: 26 avg loss 0.35
epoch: 27 avg loss 0.33
epoch: 28 avg loss 0.33
epoch: 29 avg loss 0.33
epoch: 30 avg loss 0.34
epoch: 31 avg loss 0.34
epoch: 32 avg loss 0.33
epoch: 33 avg loss 0.32
epoch: 34 avg loss 0.33
epoch: 35 avg loss 0.33
epoch: 36 avg loss 0.31
epoch: 37 avg loss 0.31
epoch: 38 avg loss 0.34
epoch: 39 avg loss 0.30
epoch: 40 avg loss 0.29
epoch: 41 avg loss 0.31
ep

In [29]:
torch.save(model.state_dict(), "10_31_2.pth")

In [30]:
model.state_dict()

OrderedDict([('features.0.0.weight',
              tensor([[[[ 0.0111,  0.0524,  0.2606],
                        [-0.2092, -0.1338,  0.1465],
                        [-0.0704, -0.2544, -0.0135]]],
              
              
                      [[[-0.1751,  0.2248, -0.0237],
                        [ 0.0573, -0.2224,  0.1846],
                        [-0.2693, -0.3187, -0.1637]]],
              
              
                      [[[-0.1354,  0.1638,  0.3037],
                        [-0.1084, -0.2364, -0.1252],
                        [-0.0553,  0.0251, -0.1426]]],
              
              
                      [[[ 0.1337, -0.0077, -0.2621],
                        [-0.3046,  0.2932,  0.2450],
                        [-0.1187,  0.2944,  0.2961]]],
              
              
                      [[[ 0.0381, -0.2808, -0.2079],
                        [-0.2697,  0.3243, -0.2935],
                        [ 0.1715, -0.0540,  0.2164]]],
              
              
        

In [31]:
np.sqrt(0.22/16)

0.11726039399558574